In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 


In [2]:
base_path = 'E:\\indoor-location-navigation\\'

In [3]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv(base_path + 'sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [5]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train\\'+building+'\\*')))
    print(building)
    wifi = list()
    for folder in folders:
        print(folder)
        floor = floor_map[folder.split('\\')[-1]]
        print(floor)
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 1000].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
E:\indoor-location-navigation\train\5a0546857ecc773753327266\B1
-1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F1
0
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F2
1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F3
2
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F4
3
941
5c3c44b80379370013e0fd2b
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\B1
-1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F1
0
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F2
1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F3
2
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F4
3
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F5
4
867
5d27075f03f801723c2e360f
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\B1
-1
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\F1
0
E:\indoor-location-navigation\train\5d27075f03f801723c2e

1105
5da958dd46f8266d0737457b
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\B1
-1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F1
0
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F2
1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F3
2
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F4
3
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F5
4
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F6
5
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F7
6
1098
5dbc1d84c1eb61796cf7c010
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\B1
-1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F2
1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F3
2
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F4
3
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F5
4
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F6
5
E:\indoor-loca

In [6]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [11]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('\\')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            wifi = list()
            waypoint = list()
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)

            df = pd.DataFrame(np.array(wifi))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).replace(np.nan, -999).T
                feat["x"] = float(waypoint[nearest_wp_index][2])
                feat["y"] = float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('/')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [22]:
ssubm_building_g = ssubm_df.groupby(0) #Grouped by building
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
#     print(gid0, 'gid0')
#     print(g0, 'g0')
#     print()
    for gid,g in g0.groupby(1):
        # get all wifi time locations, 
#         print(gid, 'gid')
#         print(g, 'g')
#         print()
        with open(os.path.join(base_path, 'test\\' + g.iloc[0,1] + '.txt'), encoding="utf8") as f:
            txt = f.readlines()

        wifi = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)

        wifi_df = pd.DataFrame(wifi) #All wifi points for certain path
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist()) #Timestamps for wifi for certain path
        #print(wifi_points)
        for timepoint in g.iloc[:,2].tolist():
            deltas = (wifi_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df

In [27]:
import xgboost as xgb

In [28]:
feature_path = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\'

In [29]:
train_files = sorted(glob.glob(os.path.join(feature_path, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_path, '*_test.csv')))

In [96]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    
    data = data.sample(frac=1)
    x_train = data.iloc[:int(len(data)*0.75),:-4]
    y_trainy = data.iloc[:int(len(data)*0.75),-3]
    y_trainx = data.iloc[:int(len(data)*0.75),-4]
    y_trainf = data.iloc[:int(len(data)*0.75),-2] + 2 #To make floors classes from 0 to 10
    
    x_test = data.iloc[int(len(data)*0.75):,:-4]
    y_testy = data.iloc[int(len(data)*0.75):,-3]
    y_testx = data.iloc[int(len(data)*0.75):,-4]
    y_testf = data.iloc[int(len(data)*0.75):,-2] + 2 #To make floors classes from 0 to 10
    
    
    dtrainX = xgb.DMatrix(x_train, label=y_trainx)
    dtestX = xgb.DMatrix(x_test, label=y_testx)
    
    dtrainY = xgb.DMatrix(x_train, label=y_trainy)
    dtestY = xgb.DMatrix(x_test, label=y_testy)
    
    dtrainF = xgb.DMatrix(x_train, label=y_trainf)
    dtestF = xgb.DMatrix(x_test, label=y_testf)
    
    paramsXY = {
    # Parameters that we are going to tune.
    'max_depth':60,
    'min_child_weight': 1,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.5,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'mae',
    'booster':'dart',
    'random_state':17,
    'verbosity':1,
    'n_jobs':-1
    }
    
    paramsF = {
    # Parameters that we are going to tune.
    'max_depth':60,
    'min_child_weight': 1,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.5,
    # Other parameters
    'objective':'multi:softmax',
    'eval_metric':'merror',
    'num_class':11,
    'booster':'gbtree',
    'random_state':17,
    'verbosity':1,
    'n_jobs':-1
    }
    
    num_boost_round=999
    
    xgb_rX = xgb.train(
    paramsXY,
    dtrainX,
    num_boost_round=num_boost_round,
    evals=[(dtestX, "Test")],
    early_stopping_rounds=60
    )
    
    xgb_rY = xgb.train(
    paramsXY,
    dtrainY,
    num_boost_round=num_boost_round,
    evals=[(dtestY, "Test")],
    early_stopping_rounds=60
    )
    
    xgb_cF = xgb.train(
    paramsF,
    dtrainF,
    num_boost_round=num_boost_round,
    evals=[(dtestF, "Test")],
    early_stopping_rounds=60
    )
    
#     modely = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modely.fit(x_train, y_trainy)

#     modelx = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modelx.fit(x_train, y_trainx)

#     modelf = lgb.LGBMClassifier(
#         n_estimators=125, num_leaves=90)
#     modelf.fit(x_train, y_trainf)
    
    test_predsx = xgb_rX.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsy = xgb_rY.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsf = xgb_cF.predict(xgb.DMatrix(test_data.iloc[:,:-1]+2))
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns[1:]
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)-2 #Returning floor classes back to original
    predictions.append(test_preds)

[0]	Test-mae:79.96051
[1]	Test-mae:64.18188
[2]	Test-mae:51.55757
[3]	Test-mae:41.47234
[4]	Test-mae:33.38947
[5]	Test-mae:26.98509
[6]	Test-mae:21.85614
[7]	Test-mae:17.78715
[8]	Test-mae:14.53086
[9]	Test-mae:11.99084
[10]	Test-mae:9.98102
[11]	Test-mae:8.41313
[12]	Test-mae:7.20636
[13]	Test-mae:6.27802
[14]	Test-mae:5.56948
[15]	Test-mae:5.03812
[16]	Test-mae:4.62828
[17]	Test-mae:4.31516
[18]	Test-mae:4.08629
[19]	Test-mae:3.92109
[20]	Test-mae:3.79706
[21]	Test-mae:3.70534
[22]	Test-mae:3.63383
[23]	Test-mae:3.58209
[24]	Test-mae:3.54596
[25]	Test-mae:3.51221
[26]	Test-mae:3.48600
[27]	Test-mae:3.46629
[28]	Test-mae:3.45153
[29]	Test-mae:3.43873
[30]	Test-mae:3.42787
[31]	Test-mae:3.42103
[32]	Test-mae:3.41445
[33]	Test-mae:3.40899
[34]	Test-mae:3.40435
[35]	Test-mae:3.40064
[36]	Test-mae:3.39806
[37]	Test-mae:3.39522
[38]	Test-mae:3.39291
[39]	Test-mae:3.39089
[40]	Test-mae:3.38935
[41]	Test-mae:3.38797
[42]	Test-mae:3.38693
[43]	Test-mae:3.38584
[44]	Test-mae:3.38494
[45]	Test-

[39]	Test-merror:0.00043
[40]	Test-merror:0.00043
[41]	Test-merror:0.00043
[42]	Test-merror:0.00043
[43]	Test-merror:0.00043
[44]	Test-merror:0.00043
[45]	Test-merror:0.00043
[46]	Test-merror:0.00043
[47]	Test-merror:0.00043
[48]	Test-merror:0.00043
[49]	Test-merror:0.00043
[50]	Test-merror:0.00043
[51]	Test-merror:0.00043
[52]	Test-merror:0.00043
[53]	Test-merror:0.00043
[54]	Test-merror:0.00043
[55]	Test-merror:0.00043
[56]	Test-merror:0.00043
[57]	Test-merror:0.00043
[58]	Test-merror:0.00043
[59]	Test-merror:0.00043
[60]	Test-merror:0.00043
[61]	Test-merror:0.00043
[62]	Test-merror:0.00043
[63]	Test-merror:0.00043
[64]	Test-merror:0.00043
[65]	Test-merror:0.00043
[66]	Test-merror:0.00043
[67]	Test-merror:0.00043
[68]	Test-merror:0.00043
[0]	Test-mae:182.77736
[1]	Test-mae:146.42035
[2]	Test-mae:117.33434
[3]	Test-mae:93.97753
[4]	Test-mae:75.29309
[5]	Test-mae:60.34021
[6]	Test-mae:48.38423
[7]	Test-mae:38.81907
[8]	Test-mae:31.21336
[9]	Test-mae:25.13942
[10]	Test-mae:20.36491
[11]

[173]	Test-mae:3.03262
[174]	Test-mae:3.03262
[175]	Test-mae:3.03262
[176]	Test-mae:3.03262
[177]	Test-mae:3.03262
[178]	Test-mae:3.03262
[179]	Test-mae:3.03262
[180]	Test-mae:3.03262
[181]	Test-mae:3.03262
[182]	Test-mae:3.03262
[183]	Test-mae:3.03262
[184]	Test-mae:3.03262
[185]	Test-mae:3.03262
[186]	Test-mae:3.03262
[187]	Test-mae:3.03262
[188]	Test-mae:3.03262
[189]	Test-mae:3.03262
[190]	Test-mae:3.03262
[191]	Test-mae:3.03262
[192]	Test-mae:3.03262
[193]	Test-mae:3.03262
[194]	Test-mae:3.03262
[195]	Test-mae:3.03262
[196]	Test-mae:3.03262
[197]	Test-mae:3.03262
[198]	Test-mae:3.03262
[199]	Test-mae:3.03262
[200]	Test-mae:3.03262
[201]	Test-mae:3.03262
[202]	Test-mae:3.03262
[203]	Test-mae:3.03262
[204]	Test-mae:3.03262
[205]	Test-mae:3.03262
[206]	Test-mae:3.03262
[207]	Test-mae:3.03262
[208]	Test-mae:3.03262
[209]	Test-mae:3.03262
[210]	Test-mae:3.03262
[211]	Test-mae:3.03262
[212]	Test-mae:3.03262
[213]	Test-mae:3.03262
[214]	Test-mae:3.03262
[215]	Test-mae:3.03262
[216]	Test-

[228]	Test-mae:3.01636
[229]	Test-mae:3.01636
[230]	Test-mae:3.01636
[231]	Test-mae:3.01636
[232]	Test-mae:3.01636
[233]	Test-mae:3.01636
[234]	Test-mae:3.01636
[235]	Test-mae:3.01636
[236]	Test-mae:3.01636
[237]	Test-mae:3.01636
[238]	Test-mae:3.01636
[239]	Test-mae:3.01636
[240]	Test-mae:3.01636
[241]	Test-mae:3.01636
[242]	Test-mae:3.01636
[243]	Test-mae:3.01636
[244]	Test-mae:3.01636
[245]	Test-mae:3.01636
[246]	Test-mae:3.01636
[247]	Test-mae:3.01636
[248]	Test-mae:3.01636
[249]	Test-mae:3.01636
[250]	Test-mae:3.01636
[251]	Test-mae:3.01636
[252]	Test-mae:3.01636
[253]	Test-mae:3.01636
[254]	Test-mae:3.01636
[255]	Test-mae:3.01636
[256]	Test-mae:3.01636
[257]	Test-mae:3.01636
[258]	Test-mae:3.01636
[259]	Test-mae:3.01636
[260]	Test-mae:3.01636
[261]	Test-mae:3.01636
[262]	Test-mae:3.01636
[263]	Test-mae:3.01636
[264]	Test-mae:3.01636
[265]	Test-mae:3.01636
[266]	Test-mae:3.01636
[267]	Test-mae:3.01636
[268]	Test-mae:3.01636
[269]	Test-mae:3.01636
[270]	Test-mae:3.01636
[271]	Test-

[284]	Test-mae:2.50263
[285]	Test-mae:2.50263
[286]	Test-mae:2.50263
[287]	Test-mae:2.50263
[288]	Test-mae:2.50263
[289]	Test-mae:2.50263
[290]	Test-mae:2.50263
[291]	Test-mae:2.50263
[292]	Test-mae:2.50263
[293]	Test-mae:2.50263
[294]	Test-mae:2.50263
[295]	Test-mae:2.50263
[296]	Test-mae:2.50263
[297]	Test-mae:2.50263
[298]	Test-mae:2.50263
[299]	Test-mae:2.50263
[300]	Test-mae:2.50263
[301]	Test-mae:2.50263
[302]	Test-mae:2.50263
[303]	Test-mae:2.50263
[304]	Test-mae:2.50263
[305]	Test-mae:2.50263
[306]	Test-mae:2.50263
[307]	Test-mae:2.50263
[308]	Test-mae:2.50263
[309]	Test-mae:2.50263
[310]	Test-mae:2.50263
[311]	Test-mae:2.50263
[312]	Test-mae:2.50263
[313]	Test-mae:2.50263
[314]	Test-mae:2.50263
[315]	Test-mae:2.50263
[316]	Test-mae:2.50263
[317]	Test-mae:2.50263
[0]	Test-merror:0.01758
[1]	Test-merror:0.00642
[2]	Test-merror:0.00490
[3]	Test-merror:0.00406
[4]	Test-merror:0.00372
[5]	Test-merror:0.00253
[6]	Test-merror:0.00203
[7]	Test-merror:0.00203
[8]	Test-merror:0.00169
[9

...

[122]	Test-mae:4.53740
[123]	Test-mae:4.53739
[124]	Test-mae:4.53739
[125]	Test-mae:4.53740
[126]	Test-mae:4.53739
[127]	Test-mae:4.53737
[128]	Test-mae:4.53737
[129]	Test-mae:4.53738
[130]	Test-mae:4.53741
[131]	Test-mae:4.53744
[132]	Test-mae:4.53740
[133]	Test-mae:4.53742
[134]	Test-mae:4.53742
[135]	Test-mae:4.53742
[136]	Test-mae:4.53744
[137]	Test-mae:4.53743
[138]	Test-mae:4.53742
[139]	Test-mae:4.53743
[140]	Test-mae:4.53741
[141]	Test-mae:4.53740
[142]	Test-mae:4.53738
[143]	Test-mae:4.53738
[144]	Test-mae:4.53737
[145]	Test-mae:4.53737
[146]	Test-mae:4.53738
[147]	Test-mae:4.53737
[148]	Test-mae:4.53737
[149]	Test-mae:4.53736
[150]	Test-mae:4.53735
[151]	Test-mae:4.53736
[152]	Test-mae:4.53735
[153]	Test-mae:4.53735
[154]	Test-mae:4.53734
[155]	Test-mae:4.53735
[156]	Test-mae:4.53735
[157]	Test-mae:4.53735
[158]	Test-mae:4.53735
[159]	Test-mae:4.53735
[160]	Test-mae:4.53737
[161]	Test-mae:4.53737
[162]	Test-mae:4.53738
[163]	Test-mae:4.53738
[164]	Test-mae:4.53737
[165]	Test-

[192]	Test-mae:2.25087
[193]	Test-mae:2.25087
[194]	Test-mae:2.25087
[195]	Test-mae:2.25087
[196]	Test-mae:2.25087
[197]	Test-mae:2.25087
[198]	Test-mae:2.25087
[199]	Test-mae:2.25087
[200]	Test-mae:2.25087
[201]	Test-mae:2.25087
[202]	Test-mae:2.25087
[203]	Test-mae:2.25087
[204]	Test-mae:2.25087
[205]	Test-mae:2.25087
[206]	Test-mae:2.25087
[207]	Test-mae:2.25087
[208]	Test-mae:2.25087
[209]	Test-mae:2.25087
[210]	Test-mae:2.25087
[211]	Test-mae:2.25087
[212]	Test-mae:2.25087
[213]	Test-mae:2.25087
[214]	Test-mae:2.25087
[215]	Test-mae:2.25087
[216]	Test-mae:2.25087
[217]	Test-mae:2.25087
[218]	Test-mae:2.25087
[219]	Test-mae:2.25087
[220]	Test-mae:2.25087
[221]	Test-mae:2.25087
[222]	Test-mae:2.25087
[223]	Test-mae:2.25087
[224]	Test-mae:2.25087
[225]	Test-mae:2.25087
[226]	Test-mae:2.25087
[227]	Test-mae:2.25087
[228]	Test-mae:2.25087
[229]	Test-mae:2.25087
[230]	Test-mae:2.25087
[231]	Test-mae:2.25087
[232]	Test-mae:2.25087
[233]	Test-mae:2.25087
[234]	Test-mae:2.25087
[0]	Test-ma

[14]	Test-mae:6.06884
[15]	Test-mae:5.37303
[16]	Test-mae:4.84528
[17]	Test-mae:4.46381
[18]	Test-mae:4.18516
[19]	Test-mae:3.98012
[20]	Test-mae:3.82260
[21]	Test-mae:3.70992
[22]	Test-mae:3.62465
[23]	Test-mae:3.56020
[24]	Test-mae:3.51111
[25]	Test-mae:3.47564
[26]	Test-mae:3.44412
[27]	Test-mae:3.42096
[28]	Test-mae:3.40333
[29]	Test-mae:3.38889
[30]	Test-mae:3.37701
[31]	Test-mae:3.36636
[32]	Test-mae:3.35850
[33]	Test-mae:3.35306
[34]	Test-mae:3.34783
[35]	Test-mae:3.34373
[36]	Test-mae:3.34061
[37]	Test-mae:3.33880
[38]	Test-mae:3.33713
[39]	Test-mae:3.33522
[40]	Test-mae:3.33307
[41]	Test-mae:3.33148
[42]	Test-mae:3.33036
[43]	Test-mae:3.32930
[44]	Test-mae:3.32824
[45]	Test-mae:3.32728
[46]	Test-mae:3.32665
[47]	Test-mae:3.32644
[48]	Test-mae:3.32600
[49]	Test-mae:3.32540
[50]	Test-mae:3.32519
[51]	Test-mae:3.32450
[52]	Test-mae:3.32404
[53]	Test-mae:3.32390
[54]	Test-mae:3.32371
[55]	Test-mae:3.32348
[56]	Test-mae:3.32321
[57]	Test-mae:3.32294
[58]	Test-mae:3.32268
[59]	Test-

[208]	Test-mae:3.33225
[209]	Test-mae:3.33225
[210]	Test-mae:3.33225
[211]	Test-mae:3.33225
[212]	Test-mae:3.33225
[213]	Test-mae:3.33225
[214]	Test-mae:3.33225
[215]	Test-mae:3.33225
[216]	Test-mae:3.33225
[217]	Test-mae:3.33225
[218]	Test-mae:3.33225
[219]	Test-mae:3.33225
[220]	Test-mae:3.33225
[221]	Test-mae:3.33225
[222]	Test-mae:3.33225
[223]	Test-mae:3.33225
[224]	Test-mae:3.33225
[225]	Test-mae:3.33225
[226]	Test-mae:3.33225
[227]	Test-mae:3.33225
[228]	Test-mae:3.33225
[229]	Test-mae:3.33225
[230]	Test-mae:3.33225
[231]	Test-mae:3.33225
[232]	Test-mae:3.33225
[233]	Test-mae:3.33225
[234]	Test-mae:3.33225
[235]	Test-mae:3.33225
[0]	Test-merror:0.02201
[1]	Test-merror:0.00544
[2]	Test-merror:0.00247
[3]	Test-merror:0.00099
[4]	Test-merror:0.00099
[5]	Test-merror:0.00099
[6]	Test-merror:0.00049
[7]	Test-merror:0.00049
[8]	Test-merror:0.00025
[9]	Test-merror:0.00025
[10]	Test-merror:0.00025
[11]	Test-merror:0.00000
[12]	Test-merror:0.00000
[13]	Test-merror:0.00000
[14]	Test-merror

[135]	Test-mae:3.79567
[136]	Test-mae:3.79568
[137]	Test-mae:3.79569
[138]	Test-mae:3.79560
[139]	Test-mae:3.79564
[140]	Test-mae:3.79562
[141]	Test-mae:3.79556
[142]	Test-mae:3.79561
[143]	Test-mae:3.79560
[144]	Test-mae:3.79564
[145]	Test-mae:3.79565
[146]	Test-mae:3.79572
[147]	Test-mae:3.79570
[148]	Test-mae:3.79578
[149]	Test-mae:3.79577
[150]	Test-mae:3.79572
[151]	Test-mae:3.79565
[152]	Test-mae:3.79562
[153]	Test-mae:3.79561
[154]	Test-mae:3.79568
[155]	Test-mae:3.79569
[156]	Test-mae:3.79573
[157]	Test-mae:3.79573
[158]	Test-mae:3.79574
[159]	Test-mae:3.79572
[160]	Test-mae:3.79579
[161]	Test-mae:3.79582
[162]	Test-mae:3.79583
[163]	Test-mae:3.79583
[164]	Test-mae:3.79580
[165]	Test-mae:3.79573
[166]	Test-mae:3.79583
[167]	Test-mae:3.79583
[168]	Test-mae:3.79584
[169]	Test-mae:3.79586
[170]	Test-mae:3.79585
[171]	Test-mae:3.79586
[172]	Test-mae:3.79583
[173]	Test-mae:3.79582
[174]	Test-mae:3.79583
[175]	Test-mae:3.79582
[176]	Test-mae:3.79581
[177]	Test-mae:3.79581
[178]	Test-

In [97]:
# generate prediction file 
all_preds = pd.concat(predictions)


In [98]:
all_preds.reset_index(inplace=True)

In [99]:
all_preds = all_preds.reindex(ssubm.index)


In [100]:
all_preds.to_csv('submission.csv', index=False)

In [101]:
all_preds

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.224716,99.557564
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,88.289108,93.667480
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,89.959503,104.976700
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.290771,106.487343
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.809952,110.089241
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,214.591034,87.356285
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,211.844345,84.988022
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,201.142822,108.069359
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,201.129257,103.072083
